In [6]:
"""实习题目2：

题目描述
对时间序列进行异常点检测：使用提供的train（长度18天）和test（长度6天）数据，设计异常检测模型，模型 类型不限，可以为有监督或无监督，对test曲线的剩余部分（14天）进行异常检测，并按数据样例的格式输出检测 结果

评价方式
precision: 若检测到t时刻为异常，寻找距离t最近的实际异常点t1，score = (5-distance(t,t1)) / 5，score小于0则 记为0
统计所有检测到的异常点的得分，求均值做为最终precision得分

recall: 若t1时刻为实际异常点，寻找距离t最近的检测到的异常点t，score = (5-distance(t,t1)) / 5，score小 于0则记为0
统计所有实际异常点的得分，求均值做为最终recall得分 3.f1

f1 = (1+1.2**2) * P * R / (P*1.2**2 + R)

数据说明
1.train 训练数据包含100个文件，每个文件中是一条长度19天的曲线数据，时间频率为1分钟，每条曲线数据彼此独立 数据有异常标签（仅供参考，可能有噪声）
2.test
验证数据包含37个文件，每个文件中是一条长度6天的曲线数据，时间频率为1分钟，每条曲线数据彼此独立 均无标签
验证数据的后14天数据，将在验收时提供

数据样例
value,time,alert_flag 171.15613,2020-05-19 00:00:00,-1 171.47934,2020-05-19 00:01:00,1 171.74709,2020-05-19 00:02:00,1
value：采样数值 time：采样时间 alert_flag：异常标志，0为正常，1为异常，-1为未知

为了方便起见

value由三个数值分段产生，将为string
time由年份月份和日期产生，将为datatime或者string
alert_flag由0，1组成，将为boolearn
"""

'实习题目2：\n\n题目描述\n对时间序列进行异常点检测：使用提供的train（长度18天）和test（长度6天）数据，设计异常检测模型，模型 类型不限，可以为有监督或无监督，对test曲线的剩余部分（14天）进行异常检测，并按数据样例的格式输出检测 结果\n\n评价方式\nprecision: 若检测到t时刻为异常，寻找距离t最近的实际异常点t1，score = (5-distance(t,t1)) / 5，score小于0则 记为0\n统计所有检测到的异常点的得分，求均值做为最终precision得分\n\nrecall: 若t1时刻为实际异常点，寻找距离t最近的检测到的异常点t，score = (5-distance(t,t1)) / 5，score小 于0则记为0\n统计所有实际异常点的得分，求均值做为最终recall得分 3.f1\n\nf1 = (1+1.2**2) * P * R / (P*1.2**2 + R)\n\n数据说明\n1.train 训练数据包含100个文件，每个文件中是一条长度19天的曲线数据，时间频率为1分钟，每条曲线数据彼此独立 数据有异常标签（仅供参考，可能有噪声）\n2.test\n验证数据包含37个文件，每个文件中是一条长度6天的曲线数据，时间频率为1分钟，每条曲线数据彼此独立 均无标签\n验证数据的后14天数据，将在验收时提供\n\n数据样例\nvalue,time,alert_flag 171.15613,2020-05-19 00:00:00,-1 171.47934,2020-05-19 00:01:00,1 171.74709,2020-05-19 00:02:00,1\nvalue：采样数值 time：采样时间 alert_flag：异常标志，0为正常，1为异常，-1为未知\n\n为了方便起见\n\nvalue由三个数值分段产生，将为string\ntime由年份月份和日期产生，将为datatime或者string\nalert_flag由0，1组成，将为boolearn\n'

In [23]:
import os, datetime as dt, math
import datetime
import numpy as np
import pandas as pd
import plotly.express as px
import featuretools as ft
import sklearn

#寻找到主路径，将主路径所有数据包（csv格式）解析过滤,排除文件中的隐藏成分
#再通过路径合并，读取文件夹中每一个文件

path = r'D:/Computer Science/Fault-Analysis/train/'
#path = r'/data/jupyterhub/notebook/dickensyang_jupyter/Fault_Analysis/train/'
files = list(filter(lambda x: x!='.ipynb_checkpoints', os.listdir(path)))
allTrainData=[]
for data in files:
    readAllCsv = pd.read_csv(path+data, index_col=None, header=0, sep=",")
    allTrainData.append(readAllCsv)
allData=pd.concat(allTrainData,ignore_index=True)

#def features(allData):
#特征一览
#特征1：时间序列
time=pd.to_datetime(allData['time'], format='%Y-%m-%d %H:%M:%S')

#特征2：两点之间的差值
valueDiff = (allData['value']-allData['value'].shift(periods=1)).dropna(axis=0, how='any')
valueDiffMean = pd.Series(valueDiff.mean())
valueDiff = valueDiffMean.append(valueDiff, ignore_index=True)
#特征3:两点之间的距离 (现在把所有时间序列间隔定义为 1)
allData['index'] = allData.index
indexDiff = (allData['index']-allData['index'].shift(periods=1)).dropna(axis=0, how='any')
indexDiffMean = pd.Series(indexDiff.mean())
indexDiff = indexDiffMean.append(indexDiff, ignore_index=True)
disDiff=(valueDiff**2+indexDiff**2).apply(math.sqrt)
#特征5：两点之间斜率
#需要把时间参数转化为可以计算的       
slope=valueDiff/indexDiff
#特征6:两点之间的正弦值和余弦值
sin = valueDiff/(valueDiff**2+indexDiff**2).apply(math.sqrt)
cos = indexDiff/(valueDiff**2+indexDiff**2).apply(math.sqrt)
#特征7:两点之间算术平均值和几何平均值
add = ((allData['value']+allData['value'].shift(periods=1))/2).dropna(axis=0, how='any')
addMean = pd.Series(add.mean())
add = addMean.append(add,ignore_index=True)
multi=((allData['value']*allData['value'].shift(periods=1)).apply(math.sqrt)).dropna(axis=0, how='any')
multiMean = pd.Series(multi.mean())
multi = multiMean.append(multi, ignore_index=True)

#特征8：两点之间同比/3天环比/7天环比
dayTodayGap=pd.DataFrame(allData['value'].shift(periods=1440).dropna(axis=0, how='any'))





,value
1440,185.81683
1441,185.41160
1442,185.35612
1443,185.13422
1444,184.56013
...,...
51698,2204.23140
51699,2205.27300
51700,2203.71070
51701,2211.00120


In [ ]:

#for allData in allTrainData:
#features(allData)

# dayToday = (allData['value']-
#             )/allData['value'].shift(periods=1440).dropna(axis=0, how='any')
# dayTodayMean = pd.Series(dayToday.mean())
# dayToday = dayTodayMean.append(dayToday, ignore_index=True)
# print(dayToday)    
#threedaytoday=all_data['value']/all_data['value'].shift(periods=4320).dropna(axis=0, how='any')
#sevendaytoday=all_data['value']/all_data['value'].shift(periods=10080).dropna(axis=0, how='any')


In [ ]:

#特征8：时间准点标记
    timeList=list(range(1,24))
    timeListDict=[]
    for H in timeList:
        if H<10:
            H='0'+ str(H)
        else:
            H=H
        timeListDict.append(int(H))

    timeListDict2=timeListDict[1:23]

    for a in timeListDict:
        startTime1=dt.datetime.strptime(f'{a}:50:00', '%H:%M:%S').time()
        endTime1=dt.datetime.strptime(f'{a}:59:00','%H:%M:%S').time()
        time_on_clock1=time[(start_time1<=time.dt.time) & (end_time1>=time.dt.time)] 

    for b in time_list_dict_n:
        start_time2=dt.datetime.strptime(f'{b}:00:00', '%H:%M:%S').time()
        end_time2=dt.datetime.strptime(f'{b}:10:00','%H:%M:%S').time()
        time_on_clock2=time[(start_time2<=time.dt.time) & (end_time2>=time.dt.time)]

    time_fix=(pd.concat([time_on_clock1,time_on_clock2],sort=True)).to_frame()

    time_fix['index']=time_fix.index
    all_data['index']=all_data.index

    if time_fix['index'].eq(all_data['index']).any()==True:
        new_alert_flag=all_data['alert_flag'].replace(to_replace=0,value=1)
        new_alert_flag=all_data['alert_flag'].replace(to_replace=1,value=1)
    else:
        new_alert_flag=all_data['alert_flag'].replace(to_replace=0,value=0)
        new_alert_flag=all_data['alert_flag'].replace(to_replace=1,value=0)

#确认完特征后，由于每个特征在绝对值上差距过多，并在后涉及距离计算，因此将通过标准化处理每个特征
    value_standard = (all_data['value']-all_data['value'].mean())/all_data['value'].std()
    value_diff_standard=(value_diff-value_diff.mean())/value_diff.std()
    dis_diff_standard=(dis_diff-dis_diff.mean())/dis_diff.std()
    slope_standard=(slope-slope.mean())/slope.std()
    sin_standard=(sin-sin.mean())/sin.std()
    cos_standard=(cos-cos.mean())/cos.std()
    add_standard=(add-add.mean())/add.std()
    multi_standard=(multi-multi.mean())/multi.std()
#     daytoday_standard=(daytoday-daytoday.mean())/daytoday.std()
#     threedaytoday_standard=(threedaytoday-threedaytoday.mean())/threedaytoday.std()
#     sevendaytoday_standard=(sevendaytoday-sevendaytoday.mean())/sevendaytoday.std()

    dict={'time':time,
          'value_standard':value_standard,
          'value_diff_standard':value_diff_standard,
          'dis_diff_standard':dis_diff_standard,
          'slope_standard':slope_standard,
          'sin_standard':sin_standard,
          'cos_standard':cos_standard,
          'add_standard':add_standard,
          'multi_standard':multi_standard,
#           'daytoday_standard':daytoday_standard,
#           'threedaytoday_standard':threedaytoday_standard,
#           'sevendaytoday_standard':sevendaytoday_standard,
          'new_alert_flag':new_alert_flag,
          'alert_flag':all_data['alert_flag']}

    new_all_data=pd.DataFrame(dict)[1:2556630].dropna()
    es=ft.EntitySet(id='feature_creation')
    es=es.entity_from_dataframe(entity_id='new_all_data',dataframe=new_all_data[['value_standard','value_diff_standard','dis_diff_standard','slope_standard','sin_standard','cos_standard','add_standard','multi_standard']], index='index', make_index=True)
    trans_primitives=['add_numeric', 'multiply_numeric', 'subtract_numeric']#'divide_numeric'  # 2列相加减乘除来生成新特征
    # max_depth=50，在原特征上进行50次运算产生新特征
    feature_matrix, feature_names = ft.dfs(entityset=es, target_entity='new_all_data', trans_primitives=trans_primitives)
    feature_matrix['new_alert_flag']=new_alert_flag
    #主成因分析法,使用mle算法
    from sklearn.decomposition import PCA
    pca=PCA(n_components='mle', svd_solver='full',random_state=10)
    data_pca=pca.fit_transform(feature_matrix)
    #通过PCA中mle模型发现个成分

    #选取被强化的特征和最初的alert_flag进行训练集和测试集分类
    from sklearn.model_selection import train_test_split
    # 标注出所有测试特征和异常点分类
    data=data_pca
    label=new_all_data['alert_flag'].tolist()
    
    #X为所有的X值，y为X值所对应的结果值。test_size=0.20表示测试集占20%
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20,random_state=10)
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    randomfor=RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_split=2, n_jobs=-1,random_state=10).fit(X_train,y_train)
    


In [ ]:

    #X为所有的X值，y为X值所对应的结果值。test_size=0.20表示测试集占20%
    X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.20,random_state=10)
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score

    randomfor=RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_split=2, n_jobs=-1,random_state=10).fit(X_train,y_train)
    return randomfor.score(X_train,y_train)

for all_data in all_train_data:
    features(all_data)

In [ ]:
# #网格测试，以确定最优参数（过于耗时，对于大体量数据不建议这么跑）
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics
# param_test1= {'n_estimators':range(100,500,50)}  
# gsearch1 = GridSearchCV(estimator = RandomForestClassifier(max_depth=10,random_state=10), param_grid=param_test1,cv=5)  
# gsearch1.fit(X_train,y_train)  
# print(gsearch1.best_params_)

# param_test2= {'max_depth':range(1,11,1),'min_samples_split':range(5,15,1)}  
# gsearch2= GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, random_state=10), param_grid = param_test2,scoring='roc_auc', cv=5)  
# gsearch2.fit(X_train,y_train)  
# print(gsearch2.best_params_) 
# # 最后确定为 100，10，5

In [ ]:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import GridSearchCV
    from sklearn import metrics

    randomfor=RandomForestClassifier(n_estimators=100,max_depth=10,min_samples_split=2, n_jobs=-1,random_state=10).fit(X_train,y_train)
    precision_randomfor=cross_val_score(randomfor, X_test, y_test, cv=5, scoring='precision')
    recall_randomfor=cross_val_score(randomfor, X_test, y_test, cv=5, scoring='recall')
    f1=np.mean(np.nan_to_num((1+1.2**2) * precision_randomfor * recall_randomfor / (precision_randomfor*1.2**2 + recall_randomfor)))
    print(precision_randomfor,recall_randomfor,f1)

In [ ]:
# XGB模型参数设置
# from sklearn.model_selection import RepeatedStratifiedKFold
# xgboosting= xgb.XGBClassifier(n_estimators=100,subsample=1,colsample_bynode=1)
# xgboosting_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
# precision_scores = cross_val_score(xgboosting, X_test, y_test, scoring='precision', cv=xgboosting_cv, n_jobs=-1)
# recall_scores = cross_val_score(xgboosting, X_test, y_test, scoring='recall', cv=xgboosting_cv, n_jobs=-1)
# f1=np.mean(np.nan_to_num((1+1.2**2) * precision_scores * recall_scores / (precision_scores*1.2**2 + recall_scores)))
# print(precision_scores,recall_scores,f1)

In [ ]:
path_test = r'/data/jupyterhub/notebook/dickensyang_jupyter/Fault_Analysis/test/'
files_test = list(filter(lambda x: x!='.ipynb_checkpoints', os.listdir(path_test)))

lists_test=[]
for data_test in files_test:
    read_test_csv = pd.read_csv(path_test+data_test, index_col=None, header=0, sep=",")
    lists_test.append(read_test_csv)

data_test=pd.concat(lists_test,ignore_index=True)

data_test['index']=data_test.index
time_test=pd.to_datetime(data_test['time'], format='%Y-%m-%d %H:%M:%S')
time_diff_test=((time_test-time_test.shift(periods=1)).dropna(axis=0, how='any')).apply(lambda x:x.total_seconds()/60)
value_diff_test = (data_test['value']-data_test['value'].shift(periods=1)).dropna(axis=0, how='any')
index_diff_test=(data_test['index']-data_test['index'].shift(periods=1)).dropna(axis=0, how='any')
dis_diff_test=(value_diff_test**2+index_diff_test**2).apply(math.sqrt)
slope_test=value_diff_test/index_diff_test
sin_test=value_diff_test/(value_diff_test**2+index_diff_test**2).apply(math.sqrt)
cos_test=index_diff_test/(value_diff_test**2+index_diff_test**2).apply(math.sqrt)
add_test=((data_test['value']+data_test['value'].shift(periods=1))/2).dropna(axis=0, how='any')
multi_test=((data_test['value']*data_test['value'].shift(periods=1)).apply(math.sqrt)).dropna(axis=0, how='any')

time_list_test=list(range(1,24))
time_list_test_dict=[]
for M in time_list_test:
    if M<10:
        M='0'+ str(M)
    else:
        M=M
    time_list_test_dict.append(int(M))
time_list_test_dict_n=time_list_test_dict[1:23]

for A in time_list_test_dict:
    start_time_test_1=dt.datetime.strptime(f'{A}:50:00', '%H:%M:%S').time()
    end_time_test_1=dt.datetime.strptime(f'{A}:59:00','%H:%M:%S').time()
    time_on_clock_test_1=time_test[(start_time_test_1<=time_test.dt.time) & (end_time_test_1>=time_test.dt.time)] 

for B in time_list_test_dict_n:
    start_time_test_2=dt.datetime.strptime(f'{B}:00:00', '%H:%M:%S').time()
    end_time_test_2=dt.datetime.strptime(f'{B}:10:00','%H:%M:%S').time()
    time_on_clock_test_2=time_test[(start_time_test_2<=time_test.dt.time) & (end_time_test_2>=time_test.dt.time)]

time_test_fix=(pd.concat([time_on_clock_test_1,time_on_clock_test_2],sort=True)).to_frame()
time_test_fix['index']=time_test_fix.index
data_test['index']=data_test.index

if time_test_fix['index'].eq(data_test['index']).any()==True:
    new_test_alert_flag=data_test['alert_flag'].replace(to_replace=0,value=1)
    new_test_alert_flag=data_test['alert_flag'].replace(to_replace=1,value=1)
else:
    new_test_alert_flag=data_test['alert_flag'].replace(to_replace=0,value=0)
    new_test_alert_flag=data_test['alert_flag'].replace(to_replace=1,value=0)

time_diff_test_standard=(time_diff_test-time_diff_test.mean())/time_diff_test.std()
value_test_standard = (data_test['value']-data_test['value'].mean())/data_test['value'].std()
value_diff_test_standard=(value_diff_test-value_diff_test.mean())/value_diff_test.std()
dis_diff_test_standard=(dis_diff_test-dis_diff_test.mean())/dis_diff_test.std()
slope_test_standard=(slope_test-slope_test.mean())/slope_test.std()
sin_test_standard=(sin_test-sin_test.mean())/sin_test.std()
cos_test_standard=(cos_test-cos_test.mean())/cos_test.std()
add_test_standard=(add_test-add_test.mean())/add_test.std()
multi_test_standard=(multi_test-multi_test.mean())/multi_test.std()

In [ ]:
dict_test={'time_test':time_test,
      'value_test_standard':value_test_standard,
      'value_diff_test_standard':value_diff_test_standard,
      'dis_diff_test_standard':dis_diff_test_standard,
      'slope_test_standard':slope_test_standard,  
      'sin_test_standard':sin_test_standard,
      'cos_test_standard':cos_test_standard,
      'add_test_standard':add_test_standard,
      'multi_test_standard':multi_test_standard,
      'new_test_alert_flag':new_test_alert_flag,
      'test_alert_flag':data_test['alert_flag']}

new_test_data=pd.DataFrame(dict_test)[1:955763].dropna()

import featuretools as ft
es_test=ft.EntitySet(id='feature_creation_test')
es_test=es_test.entity_from_dataframe(entity_id='test_data',         
                         dataframe=new_test_data[['value_test_standard','value_diff_test_standard','dis_diff_test_standard','slope_test_standard','sin_test_standard','cos_test_standard','add_test_standard','multi_test_standard']],
                         index='index',
                         make_index=True)
trans_primitives_test=['add_numeric', 'multiply_numeric', 'subtract_numeric','divide_numeric']
feature_matrix_test, feature_names_test = ft.dfs(entityset=es_test, target_entity='test_data',max_depth=100, trans_primitives=trans_primitives_test, max_features=112)
feature_matrix_test['new_test_alert_flag']=new_test_alert_flag

from sklearn.decomposition import PCA
pca_test=PCA(n_components='mle', svd_solver='full',random_state=10)
data_pca_test=pca_test.fit_transform(feature_matrix_test)

data_final=data_pca_test
label_final=randomfor.predict(data_final)
label_final=np.insert(label_final,0,[0])
data_test['alert_flag']=label_final
data_test.to_csv (path_or_buf=r'/data/jupyterhub/notebook/dickensyang_jupyter/Fault_Analysis/test_result/test')

In [ ]:
import csv
import os
 
path = '../development (4).csv'
 
workspace = '../workspace'
 
with open(path, 'r', newline='') as file:
    csvreader = csv.reader(file)
    a = next(csvreader)
    print(a)
    i = j = 0
    for row in csvreader:
        print(row)
        print(f'i is {i}, j is {j}')
        # 每128个就j加1， 然后就有一个新的文件名
        if i % 128 == 0:
            j += 1
            print(f"csv {j} 生成成功")
        #csv_path = os.path.join('../new_csv_file/', 'development (4)/' + str(j) + '.csv')
        csv_path = os.path.join(workspace, 'part_{}.csv'.format(j))
        # print('/'.join(path.split('/')[:-1]))
        print(csv_path)
        # 不存在此文件的时候，就创建
        if not os.path.exists(os.path.dirname(csv_path)):
            os.makedirs(os.path.dirname(csv_path))
            with open(csv_path, 'w', newline='') as file:
                csvwriter = csv.writer(file)
                #csvwriter.writerow(['image_url'])
                csvwriter.writerow(row)
            i += 1
        # 存在的时候就往里面添加
        else:
            with open(csv_path, 'a', newline='') as file:
                csvwriter = csv.writer(file)
                csvwriter.writerow(row)
            i += 


In [ ]:
px.scatter(all_data[0:85221], x=all_data['time'][0:85221],y=all_data['value'][0:85221],color=all_data['alert_flag'][0:85221])